In [ ]:
import tess_cpm
import numpy as np
import matplotlib.pyplot as plt
import lightkurve as lk
from astropy.io import fits
from astropy.wcs import WCS

In [ ]:
def cpm_periodogram(fits_file, t_row=50, t_col=50):
    cpm = tess_cpm.CPM(fits_file, remove_bad=True)
    cpm.set_target(t_row, t_col)
    cpm.set_exclusion(10)
    cpm.set_predictor_pixels(256, method='cosine_similarity')
    cpm.lsq(0.1, rescale=True, polynomials=False)
    tess_cpm.summary_plot(cpm, 10)
    aperture_lc, lc_matrix = cpm.get_aperture_lc(box=2, show_pixel_lc=True, show_aperture_lc=True)
    lc = lk.LightCurve(time=cpm.time, flux=aperture_lc)
    pg = lc.to_periodogram(oversample_factor=500)
    fig, axs = plt.subplots(2, 1, figsize=(15, 8))
    pg.plot(ax=axs[0], c='k')
    pg.plot(ax=axs[1], c='k', view='period')
    fig.suptitle("Periodogram", fontsize=20, y=0.95)
    period = pg.period_at_max_power
    print(f"Max Power Period: {period}")
    lc.fold(period.value*4).scatter()
    plt.title(f"Folded Lightcurve with Period: {period:.4f}", fontsize=20)
    return cpm

In [ ]:
fits_file = "eclipsing_binaries/tess-s0005-1-1_65.170600_-1.740100_100x100_astrocut.fits"
# fits_file = "tess-s0002-4-2_83.272000_-69.508733_64x64_astrocut.fits"

In [ ]:
cpm_periodogram(fits_file)

In [ ]:
cpm = tess_cpm.CPM(fits_file, remove_bad=True)

In [ ]:
cpm.set_poly_model(1, 4, 2)
cpm.set_target(50, 50)
# cpm.set_target(16, 29)
# cpm.set_target(34, 71)  # Bright region bottom-right
# cpm.set_target(36, 11)
cpm.set_exclusion(10)
cpm.set_predictor_pixels(256, method='cosine_similarity')

In [ ]:
plt.figure(figsize=(12, 8))
n = 10
# locs, mask = c.get_contributing_pixels(n)
# print(c.rescaled_predictor_pixels_fluxes.shape)
# print(locs[2])
# plt.plot(c.time, c.rescaled_im_fluxes[:, locs[0][0], locs[0][1]], '-')
# plt.plot(c.time, c.rescaled_predictor_pixels_fluxes[:, 4])
for i in range(n):
    plt.figure(figsize=(12, 8))
    t = cpm.rescaled_target_fluxes
    p = cpm.rescaled_predictor_pixels_fluxes[:, i]
    plt.plot(cpm.time, t, '.')
    plt.plot(cpm.time, p, '-', alpha=0.4)
    plt.title(np.dot(p, t)/(np.linalg.norm(p)*np.linalg.norm(t)), fontsize=25)
    plt.show();

In [ ]:
cpm.lsq(0.1, rescale=True, polynomials=True)

In [ ]:
tess_cpm.summary_plot(cpm, 20, subtract_polynomials=True, save=True)

In [ ]:
cpm.sigma_clip_process(sigma=3)

In [ ]:
tess_cpm.summary_plot(cpm, 20, subtract_polynomials=True)

In [ ]:
plt.figure(figsize=(15, 8))

bgd_subtracted = cpm.rescaled_target_fluxes - cpm.lsq_prediction
plt.plot(cpm.time[cpm.valid], bgd_subtracted[cpm.valid], ".", color="black", label="Data")
# plt.plot(cpm.time, bgd_subtracted, ".-", color="C2", label="Raw")

for dump in cpm.dump_times:
    plt.axvline(dump, color="red", alpha=0.5)
plt.legend();

In [ ]:
plt.figure(figsize=(15, 8))

plt.plot(cpm.time, cpm.rescaled_target_fluxes - cpm.lsq_prediction, 
         "-o", color="C0", label="Corrected", alpha=0.5)
plt.plot(cpm.time, cpm.rescaled_target_fluxes, ".-", color="black", label="Data")
for dump in cpm.dump_times:
    plt.axvline(dump, color="red", alpha=0.5)
plt.legend();

In [ ]:
tess_cpm.summary_plot(cpm, 20, subtract_polynomials=True)

In [ ]:
cpm.sigma_clip(5.0, 6)
tess_cpm.summary_plot(cpm, 20, subtract_polynomials=True)

In [ ]:
plt.figure(figsize=(15, 6))
orig_model = np.dot(cpm.orig_m, cpm.lsq_params)
# model = np.dot(cpm.m, cpm.lsq_params)
plt.plot(cpm.orig_time, cpm.rescaled_target_fluxes - orig_model, ".-")
for dump in cpm.dump_times:
    plt.axvline(dump, color="red", alpha=0.5)
# plt.plot(cpm.time, cpm.rescaled_im_fluxes[:, cpm.target_row, cpm.target_col] - model)
# plt.plot(c.orig_time, model)

In [ ]:
cpm.sigma_clip(3.0, 3)
tess_cpm.summary_plot(cpm, 20, subtract_polynomials=False)

In [ ]:
plt.figure(figsize=(15, 6))
orig_model = np.dot(cpm.orig_m, cpm.lsq_params)
model = np.dot(cpm.m, cpm.lsq_params)
plt.plot(cpm.orig_time, cpm.orig_rescaled_im_fluxes[:, cpm.target_row, cpm.target_col] - orig_model, ".-")
for dump in cpm.dump_times:
    plt.axvline(dump, color="red", alpha=0.5)
# plt.plot(cpm.time, cpm.rescaled_im_fluxes[:, cpm.target_row, cpm.target_col] - model)
# plt.plot(c.orig_time, model)

In [ ]:
plt.plot(cpm.sigma_clipped_time, cpm.sigma_clipped_fluxes, ".-")

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(cpm.time, cpm.target_fluxes, ".-", color="k")
for dump in cpm.dump_times:
    plt.axvline(dump, color="red", alpha=0.5)

In [ ]:
from IPython.display import HTML
import matplotlib.animation as animation
cpm.entire_image(0.5, rescale=True, polynomials=True)
diff = cpm.im_diff
upscaled_diff = cpm.pixel_medians*cpm.im_diff

fig, axes = plt.subplots(1, 3, figsize=(18, 18))

ims = []
for i in range(0, diff.shape[0], 10):
    im1 = axes[0].imshow(cpm.im_fluxes[i], origin="lower", animated=True,
                        vmin=np.percentile(cpm.im_fluxes[0], 10), vmax=np.percentile(cpm.im_fluxes[0], 90))
    im2 = axes[1].imshow(diff[i], origin="lower", animated=True,
                   vmin=np.percentile(diff, 1), vmax=np.percentile(diff, 99));
    im3 = axes[2].imshow(upscaled_diff[i], origin="lower", animated=True,
                   vmin=np.percentile(upscaled_diff, 1), vmax=np.percentile(upscaled_diff, 99));
    ims.append([im1, im2, im3]);
fig.colorbar(im1, ax=axes[0], fraction=0.046)
fig.colorbar(im2, ax=axes[1], fraction=0.046)
fig.colorbar(im3, ax=axes[2], fraction=0.046)
    
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000);

HTML(ani.to_jshtml())


In [ ]:
plt.rcParams['figure.figsize'] = (12, 8)
cpm = tess_cpm.CPM(fits_file, remove_bad=True)
cpm.set_poly_model(1, 4, 2)
cpm.set_target(50, 50)
# cpm.set_target(16, 29)
# cpm.set_target(34, 71)  # Bright region bottom-right
# cpm.set_target(36, 11)
cpm.set_exclusion(10)
cpm.set_predictor_pixels(256, method='cosine_similarity')
cpm.xval(0.5)